In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py

Using TensorFlow backend.


## 对图片进行分类

In [2]:

train_path = 'dogs-vs-cats/train1/'
test_path = 'dogs-vs-cats/test1/'

dog_path = train_path + 'dog/'
cat_path = train_path + 'cat/'

if not os.path.exists(dog_path):
    names= os.listdir(train_path)
    os.mkdir(dog_path)
    os.mkdir(cat_path)
    [shutil.move(train_path + name, cat_path + name) for name in names if name.startswith('cat')]
    [shutil.move(train_path + name, dog_path + name) for name in names if name.startswith('dog')]
    


In [3]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    #进行这一步时会从Keras网站中下载imagenet模型，如果网速慢或缺乏连接，
    #可以直接在上面的连接下载，放到C:\Users\用户名\.keras\models 下
    #（模型有可能不是最新的，导致进行到这一步仍然报错），
    #或者直接点击报错的GitHub链接进行下载
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_path = "dogs-vs-cats/train1/"
    test_path = 'dogs-vs-cats/test1/'
    train_generator = gen.flow_from_directory(train_path, image_size, shuffle=False,
                                              batch_size=1)
    test_generator = gen.flow_from_directory(test_path, image_size, shuffle=False,
                                             batch_size=1, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_{0}.h5".format(base_model.name)) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [4]:
write_gap(ResNet50, (224, 224))


W0320 18:28:56.875257  4688 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\tensorflow\python\training\moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 200 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [5]:
write_gap(InceptionV3, (299, 299), inceptionV3_pre)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [6]:
write_gap(Xception, (299, 299), xception_pre)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [7]:
np.random.seed(2017)

X_train = []
X_test = []
for filenames in ["gap_resnet50.h5", "gap_inception_v3.h5", "gap_xception.h5"]:
    filename = filenames
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)


In [8]:
inputs = Input(X_train.shape[1:])#shape=（2048*3，）
x = Dropout(0.5)(inputs)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


W0320 18:58:33.689949  4688 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=1, validation_split=0.2,verbose=2)

y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 160 samples, validate on 40 samples
Epoch 1/1
 - 6s - loss: 0.5888 - acc: 0.6688 - val_loss: 1.4969 - val_acc: 0.0000e+00
100/100 [==============================] - ETA:  - 1s 15ms/step


In [11]:
y_pred

array([[0.13094783],
       [0.05426881],
       [0.03095853],
       [0.03443813],
       [0.26116753],
       [0.01878142],
       [0.0219692 ],
       [0.02792978],
       [0.00973615],
       [0.1806455 ],
       [0.33995742],
       [0.02947286],
       [0.1864104 ],
       [0.03422177],
       [0.2920377 ],
       [0.04326177],
       [0.26537746],
       [0.26114786],
       [0.01333237],
       [0.11783651],
       [0.25961027],
       [0.04270884],
       [0.10249051],
       [0.1449669 ],
       [0.20038271],
       [0.4993934 ],
       [0.11606357],
       [0.40339655],
       [0.02696654],
       [0.02847821],
       [0.01973817],
       [0.0117847 ],
       [0.01580504],
       [0.1474472 ],
       [0.21703652],
       [0.02244088],
       [0.24330959],
       [0.29098642],
       [0.42505878],
       [0.3326541 ],
       [0.05284646],
       [0.13421592],
       [0.03722295],
       [0.21565998],
       [0.37879962],
       [0.0176042 ],
       [0.03374287],
       [0.038

In [10]:
df = pd.read_csv("dogs-vs-cats/sample_submission.csv")

gen = ImageDataGenerator()
test_path = 'dogs-vs-cats/test1/'
test_generator = gen.flow_from_directory(test_path, (224, 224), shuffle=False,
                                         batch_size=1, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)


Found 100 images belonging to 1 classes.


C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,id,label
0,1,0.130948
1,2,0.186410
2,3,0.144967
3,4,0.217037
4,5,0.017604
5,6,0.019730
6,7,0.033361
7,8,0.016773
8,9,0.018776
9,10,0.054269
